# Window-функции и инструкция OVER

_Инструкция OVER_ определяет окно или набор строк внутри результирующего набора запроса. 

_Оконная функция_ вычисляет значение для каждой строки в окне.

_OVER_ можно использовать вместе с оконными функциями для вычисления статистических значений (например, для вычисления скользящих средних, суммарных статистических выражений, промежуточных итогов или первых N результатов в группе).

Оконные функции можно разделить на следующие группы:

 - Ранжирующие функции

 - Агрегатные функции

 - Аналитические функции

**Синтаксис**:

https://docs.microsoft.com/ru-RU/sql/t-sql/queries/select-over-clause-transact-sql?view=aps-pdw-2016


В одном запросе с одним предложением _FROM_ может использоваться несколько оконных функций. Предложение _OVER_ для каждой функции может отличаться в части секционирования и упорядочения.

Если _PARTITION BY_ не указан, функция обрабатывает все строки результирующего набора запроса как одну группу.

**Ранжирующие функции**

 - ROW_NUMBER
 - RANK
 - DENSE_RANK
 - NTILE

 Пример:

In [1]:
-- насетапим данные
drop table if exists #table

create table #table
(
	firstname varchar(20) not null,
	lastname varchar(20) not null,
	birthdate date not null,
	loaddate datetime not null default (getdate())
)
go

insert into #table (firstname, lastname, birthdate)
values ('Timofey', 'Gavrilenko', '1980-04-12'),
	   ('Olga', 'Gavrilenko', '1980-02-08'),
	   ('Elena', 'Bykova', '1981-10-09')

insert into #table (firstname, lastname, birthdate)
values ('Olga', 'Gavrilenko', '1980-02-08'),
	   ('Elena', 'Bykova', '1981-10-09'),
       ('Vitaliy', 'Bykov', '1980-03-12')

insert into #table (firstname, lastname, birthdate)
values ('Ilya', 'Gavrilenko', '2006-06-20'),
	   ('Timofey', 'Gavrilenko', '2007-10-20'),
       ('Anna', 'Gavrilenko', '2009-11-11'),
       ('Daria', 'Bykova', '2005-12-25'),
       ('Ksenia', 'Bykova', '2010-01-07')

select * from #table      


(3 rows affected)

(3 rows affected)

(5 rows affected)

(11 rows affected)

Total execution time: 00:00:00.0112343

firstname,lastname,birthdate,loaddate
Timofey,Gavrilenko,1980-04-12,2019-06-06 13:12:39.353
Olga,Gavrilenko,1980-02-08,2019-06-06 13:12:39.353
Elena,Bykova,1981-10-09,2019-06-06 13:12:39.353
Olga,Gavrilenko,1980-02-08,2019-06-06 13:12:39.353
Elena,Bykova,1981-10-09,2019-06-06 13:12:39.353
Vitaliy,Bykov,1980-03-12,2019-06-06 13:12:39.353
Ilya,Gavrilenko,2006-06-20,2019-06-06 13:12:39.353
Timofey,Gavrilenko,2007-10-20,2019-06-06 13:12:39.353
Anna,Gavrilenko,2009-11-11,2019-06-06 13:12:39.353
Daria,Bykova,2005-12-25,2019-06-06 13:12:39.353


In [4]:
-- Расставим людей по их возрасту
select *,
       DENSE_RANK() over(order by DATEDIFF(day, birthdate, GETDATE())) rn
from #table


(11 rows affected)

Total execution time: 00:00:00.0067381

firstname,lastname,birthdate,loaddate,rn
Ksenia,Bykova,2010-01-07,2019-06-06 13:12:39.353,1
Anna,Gavrilenko,2009-11-11,2019-06-06 13:12:39.353,2
Timofey,Gavrilenko,2007-10-20,2019-06-06 13:12:39.353,3
Ilya,Gavrilenko,2006-06-20,2019-06-06 13:12:39.353,4
Daria,Bykova,2005-12-25,2019-06-06 13:12:39.353,5
Elena,Bykova,1981-10-09,2019-06-06 13:12:39.353,6
Elena,Bykova,1981-10-09,2019-06-06 13:12:39.353,6
Timofey,Gavrilenko,1980-04-12,2019-06-06 13:12:39.353,7
Vitaliy,Bykov,1980-03-12,2019-06-06 13:12:39.353,8
Olga,Gavrilenko,1980-02-08,2019-06-06 13:12:39.353,9


В результате прошлого запроса есть дубли. Попробуем их исключить

In [27]:
-- Расставим людей по их возрасту
;with cte_group_and_enumerate
as
(
    select *,
       ROW_NUMBER() over(
                         partition by firstname, lastname, birthdate
                         order by loaddate
                        ) rn
    from #table
),
cte_deduplicate_and_renumerate
as
(
    select firstname, lastname, birthdate, loaddate
    from cte_group_and_enumerate
    where rn = 1
)
select *,
       ROW_NUMBER() over(
                         order by DATEDIFF(day, birthdate, GETDATE())
                        ) rn
from cte_deduplicate_and_renumerate

(9 rows affected)

Total execution time: 00:00:00.0059847

firstname,lastname,birthdate,loaddate,rn
Ksenia,Bykova,2010-01-07,2019-06-06 10:59:40.590,1
Anna,Gavrilenko,2009-11-11,2019-06-06 10:59:40.590,2
Timofey,Gavrilenko,2007-10-20,2019-06-06 10:59:40.590,3
Ilya,Gavrilenko,2006-06-20,2019-06-06 10:59:40.590,4
Daria,Bykova,2005-12-25,2019-06-06 10:59:40.590,5
Elena,Bykova,1981-10-09,2019-06-06 10:59:40.590,6
Timofey,Gavrilenko,1980-04-12,2019-06-06 10:59:40.590,7
Vitaliy,Bykov,1980-03-12,2019-06-06 10:59:40.590,8
Olga,Gavrilenko,1980-02-08,2019-06-06 10:59:40.590,9


Члены двух семей перемешались. Пронумеруем записи в пределах семьи


In [4]:
-- Расставим людей по их возрасту в пределах семьи. Результат сохраним во временной таблице
drop table if exists #table1

;with cte_group_and_enumerate
as
(
    select *,
       ROW_NUMBER() over(
                         partition by firstname, lastname, birthdate
                         order by loaddate
                        ) rn
    from #table
),
cte_deduplicate_and_renumerate
as
(
    select firstname, lastname, birthdate, loaddate
    from cte_group_and_enumerate
    where rn = 1
)
select *,
       ROW_NUMBER() over(
                         partition by LEFT(lastname, 5)              -- не самый лучший вариант разделения на группы
                         order by DATEDIFF(day, birthdate, GETDATE())
                        ) rn,

       into #table1
from cte_deduplicate_and_renumerate

select * from #table1

(9 rows affected)

(9 rows affected)

Total execution time: 00:00:00.0136428

firstname,lastname,birthdate,loaddate,rn
Ksenia,Bykova,2010-01-07,2019-06-06 12:22:39.463,1
Daria,Bykova,2005-12-25,2019-06-06 12:22:39.463,2
Elena,Bykova,1981-10-09,2019-06-06 12:22:39.463,3
Vitaliy,Bykov,1980-03-12,2019-06-06 12:22:39.463,4
Anna,Gavrilenko,2009-11-11,2019-06-06 12:22:39.463,1
Timofey,Gavrilenko,2007-10-20,2019-06-06 12:22:39.463,2
Ilya,Gavrilenko,2006-06-20,2019-06-06 12:22:39.463,3
Timofey,Gavrilenko,1980-04-12,2019-06-06 12:22:39.463,4
Olga,Gavrilenko,1980-02-08,2019-06-06 12:22:39.463,5


Найдем статистику распределения дней рождения по месяцам. 


In [40]:
;with cte_get_birthdate_month_and_day
as
(
    select *,
           DATEPART(month, birthdate) as month_number,
           DENSE_RANK() over(partition by DATEPART(month, birthdate) order by DATEPART(day, birthdate)) as n
    from #table1   
),
cte_month_numbers
as
(
    select CAST([value] as int) as month_number
    from OPENJSON('[1,2,3,4,5,6,7,8,9,10,11,12]') numbers
),
cte_stats
as
(
    select mn.month_number, 
           bdm.lastname,
           bdm.firstname,
           bdm.n
    from cte_month_numbers mn
    left join cte_get_birthdate_month_and_day bdm on mn.month_number = bdm.month_number
)
select * 
from cte_stats
order by month_number


(13 rows affected)

Total execution time: 00:00:00.0095824

month_number,lastname,firstname,n
1,Bykova,Ksenia,1
2,Gavrilenko,Olga,1
3,Bykov,Vitaliy,1
4,Gavrilenko,Timofey,1
5,NULL,NULL,NULL
6,Gavrilenko,Ilya,1
7,NULL,NULL,NULL
8,NULL,NULL,NULL
9,NULL,NULL,NULL
10,Bykova,Elena,1


Для каждого человека найдем два ближайших именниника (по одному в обе стороны)

In [47]:
;with cte
as
(
    select lastname,
           firstname,
           birthdate,
           LAG(lastname, 1) over(order by birthdate) as lastname_prev,
           LAG(firstname, 1) over(order by birthdate) as firstname_prev,
           LEAD(lastname, 1) over(order by birthdate) as lastname_next,
           LEAD(firstname, 1) over(order by birthdate) as firstname_next
    from #table1   
)
select * 
from cte

(9 rows affected)

Total execution time: 00:00:00.0075371

lastname,firstname,birthdate,lastname_prev,firstname_prev,lastname_next,firstname_next
Gavrilenko,Olga,1980-02-08,NULL,NULL,Bykov,Vitaliy
Bykov,Vitaliy,1980-03-12,Gavrilenko,Olga,Gavrilenko,Timofey
Gavrilenko,Timofey,1980-04-12,Bykov,Vitaliy,Bykova,Elena
Bykova,Elena,1981-10-09,Gavrilenko,Timofey,Bykova,Daria
Bykova,Daria,2005-12-25,Bykova,Elena,Gavrilenko,Ilya
Gavrilenko,Ilya,2006-06-20,Bykova,Daria,Gavrilenko,Timofey
Gavrilenko,Timofey,2007-10-20,Gavrilenko,Ilya,Gavrilenko,Anna
Gavrilenko,Anna,2009-11-11,Gavrilenko,Timofey,Bykova,Ksenia
Bykova,Ksenia,2010-01-07,Gavrilenko,Anna,NULL,NULL


Разбить множество всех записей на 3 (относительно) равных подмножества

In [14]:
;with cte_split
as
(
    select *,
       ntile(3) over(order by birthdate) as num
    from #table1
)
select * 
from cte_split


(9 rows affected)

Total execution time: 00:00:00.0258515

firstname,lastname,birthdate,loaddate,rn,num
Olga,Gavrilenko,1980-02-08,2019-06-06 12:22:39.463,5,1
Vitaliy,Bykov,1980-03-12,2019-06-06 12:22:39.463,4,1
Timofey,Gavrilenko,1980-04-12,2019-06-06 12:22:39.463,4,1
Elena,Bykova,1981-10-09,2019-06-06 12:22:39.463,3,2
Daria,Bykova,2005-12-25,2019-06-06 12:22:39.463,2,2
Ilya,Gavrilenko,2006-06-20,2019-06-06 12:22:39.463,3,2
Timofey,Gavrilenko,2007-10-20,2019-06-06 12:22:39.463,2,3
Anna,Gavrilenko,2009-11-11,2019-06-06 12:22:39.463,1,3
Ksenia,Bykova,2010-01-07,2019-06-06 12:22:39.463,1,3
